# Decision Tree algorithm from scratch

This is a **classification algorithm**, and it is used to represent all the possible solutions to a decision in a graphical way. it is much similar to the way what we used to make decisions using tree methods manually.

The procedure of building this algorithm from base is little bit long. For the implementation in this notebook, a very simple dataset has used. 

In [1]:
sample_data=[
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon']
]

Next, we can give headers for data columns. They will be useful for handling our sample dataset.

In [2]:
header=['color','diameter','label']

To implement this algorithm, lots of functions and classes are required. Let’s define them one by one. 

First, we have to define a function to get the **unique values** of a column. There are two parameters in **unique_vals** function. 
**rows** parameter represents a list of lists, and each list includes the values corresponding to a row. 
**col** is an integer parameter and it holds the index of the column which we want to get unique values. 

In [3]:
def unique_vals(rows,col):
    return set([row[col] for row in rows])

Next the count of each label has got. **class_count** is a single parameter function, and the **rows** parameter represents a list of lists which have row data. The label name and the count of it are stored in the **counts** dictionary. 

In [4]:
def class_count(rows):
    counts={}
    for row in rows: 
        label=row[-1] # in this dataset the label is always last column
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

Then we need a function to check whether a value is **numeric** or not. For that the **is_numeric** function has developed. The parameter **value** is the value which we want to check the data type. The method used in this function is the **isinstance(value, data type)**. It simply returns **True** or **False** by comparing the *value* and *data type* parameters. This is a built-in python method that can be used to verify the data type of values. 

In [6]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

When the Tree builds, the **partitioning** is an important stage. It divides the dataset. To do so, we need **questions**. The following **class** is coded for that. 

**__init__** method defines two variables called **column** and **value**. Those are the **column index** and the **feature value** respectively. Actual dataset values are going to be compared with these.

Next the **match** method compares the above two values and the corresponding actual dataset values to check whether they satisfy our conditions in the example. The two conditions that are used here, 
- If the value is *numeric* then it should be greater than the *value* defined in the class.
- If the value is *not numeric* then it should be equal to the *label* in the *column* index.


In [7]:
class Question:
    
    def __init__(self,column,value):
        self.column=column
        self.value=value
        
    def match(self,example):
        # compare the feature value in an example with the feature value in this question
        val=example[self.column]
        if is_numeric(val):
            return val>=self.value
        else:
            return val==self.value
        
    def __repr__(self):
        # just a helper to represent question in readable format
        condition='=='
        if is_numeric(self.value):
            condition='>='
        return 'Is %s %s %s?'%(header[self.column],condition,str(self.value))

Next, we can divide the dataset into partitions by matching the dataset rows with our question. If a row matches with the question, then it is added to the **true rows** list.

In [1]:
def partition(rows,question):
    
    true_rows,false_rows=[],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows,false_rows

Now our goal is finding the best split for the dataset. To do that we can use two important parameters. They are **Gini Impurity** and **Information Gain**. 

Gini impurity is a measure of correctness. It represents *the probability of classifying a row incorrectly*. To split the node, we choose the row which has **the lowest** Gini impurity.

There are two main definitions for calculating Gini impurity.

- Let’s take a dataset *T* which has samples from *k* classes. The probability of sample belonging to class *i* is *Pi*. Then the Gini impurity of the dataset *T* can be denoted as,

\begin{equation}
Gini(T) = 1 - \sum\limits_{i=1}^{k} {(P_i)^2}
\end{equation}

- The following way is also correct. 

\begin{equation}
Gini(T) = \sum\limits_{i=1}^{k} {P_i * (1 - P_i)}
\end{equation}

In [1]:
def gini(rows):
    
    counts=class_count(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl=counts[lbl]/float(len(rows))
        impurity-=prob_of_lbl**2
    return impurity

**information gain** is used to determine the homogeneous level of a branch. When we find the best split, it is important to take the branches which have the highest information gain.

In [1]:
def info_gain(left,right,current_uncertainty):
   
    # the uncertainty of the starting node, minus the weight impurity of two child nodes
    p=float(len(left)/len(right)+len(right))
    return current_uncertainty-p*gini(left)-(1-p)*gini(right)

Now, the best split can be defined using the functions, we have created so far. The duty of this function is to find the **best gain** and the **best question** to ask. 

First, we take the values of a column and send them one by one to the *Question* class, to select a question. Then the data and selected question put into the *partition* function. This process continues until both *true_rows* and *false_rows* lists get elements. After that, *gain* is calculated using *info_gain* function. If this gain value is the best value then the gain and question return as the best choices. 

In [2]:
def find_best_split(rows):
    # find the best question to ask
    best_gain=0
    best_question= None
    current_uncertainty=gini(rows)
    n_features=len(rows[0])-1
    
    for col in range(n_features):
        values=set([row[col] for row in rows])
        
        for val in values:
            question=Question(col,val)
            
            true_rows,false_rows=partition(rows,question)
            
            if len(true_rows)==0 or len(false_rows)==0:
                continue
                
            gain=info_gain(true_rows,false_rows,current_uncertainty)
            
            if gain>=best_gain:
                best_gain,best_question=gain,question
    return best_gain,best_question